1. Import Library

In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00


In [2]:
import io
import string
import numpy
import random
import sklearn.svm as svm

from sklearn.svm import LinearSVC
from collections import defaultdict
from pyvi import ViTokenizer, ViPosTagger
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.models import save_model
from sklearn.metrics import accuracy_score
import pickle

3. Tagset

In [3]:
# Tạo dictionary cho các loại từ loại và chuyển đổi ngược lại
tagsetDict = {'DET': 1,
              'INTJ': 2,
              'AUX': 3,
              'SCONJ': 4,
              'PART': 5,
              'NUM': 6,
              'CCONJ': 7,
              'VERB': 8,
              'ADJ': 9,
              'NOUN': 10,
              'SYM': 11,
              'ADP': 12,
              'ADV': 13,
              'PRON': 14,
              'PROPN': 15,
              'X': 16}
inverseTagsetDict = {tagsetDict[k]:k for k in tagsetDict}

4. Check some of the tagged words

In [4]:
# Khởi tạo các dictionary và list cho việc lưu trữ thông tin từ và bigram

wordBank = defaultdict()
bigramBank = defaultdict()
bigramDict = defaultdict(int)

bigramFreq = {x:[0]*16 for x in tagsetDict}
mostCommonBigrams = defaultdict()

prevParts = None

# Đọc dữ liệu huấn luyện và kiểm tra
f_tagged_train = io.open("ud_train.txt", encoding='utf-8').readlines()
f_tagged_test = io.open("ud_test.txt", encoding='utf-8').readlines()

train = f_tagged_train
test = f_tagged_test
print(train[:10])
print(test[:10])

['1\ttôi\tPRON\n', '2\tnhớ\tVERB\n', '3\tlời\tNOUN\n', '4\tanh\tNOUN\n', '5\tchủ_tịch\tNOUN\n', '6\txã\tNOUN\n', '7\tbùi_văn_luyến\tPROPN\n', '8\tnhắc\tVERB\n', '9\tđi\tADV\n', '10\tnhắc\tVERB\n']
['1\tthanh\tPROPN\n', '2\tbắt_chuyện\tVERB\n', '3\tvới\tADP\n', '4\thùng\tPROPN\n', '5\tvà\tCCONJ\n', '6\tnói\tVERB\n', '7\t:\tPUNCT\n', '8\t"\tPUNCT\n', '9\ttôi\tPRON\n', '10\ttrông\tVERB\n']


In [5]:
print(len(f_tagged_train))
print(len(f_tagged_test))

46378
11692


In [6]:
sentences = []
first = 1
sentence = []
for line in train:
  try:
    i, word, pos = line.split('\n')[0].split('\t')
  except:
    continue
  i = int(i)
  if i == 1:
    if first == 1:
      first = 0
    else:
      sentences.append(sentence)
      sentence = []
  part = [word, pos]
  sentence.append(part)



In [7]:
# Phân tích dữ liệu huấn luyện để tạo từ loại và bigrams
for line in sentences:
  for i, w in enumerate(line):
    parts = w

    if i >=1:
      prevParts = line[i-1]
    if len(parts) == 1 or \
      parts[1] not in tagsetDict or parts[1] == 'X':
      continue

    if i >=2 and prevParts[1] in string.punctuation:
      prevParts = line[i - 1]
    word = parts[0]
    pos = parts[1]

    if i >= 1:
      prevWord = prevParts[0]
      prevPos = prevParts[1]
      bigramBank[word] = (pos, prevWord, prevPos)
      bigramDict[(pos, prevPos)] += 1
    if word not in wordBank:
      wordBank[word] = [pos]
    else:
      wordBank[word] +=[pos]

In [8]:
# Tạo các list chứa bi-grams, tri-grams, qua-grams từ dữ liệu
bi_grams = []
tri_grams = []
qua_grams = []

for sentence in set(wordBank):
  temp = 0
  for s in list(sentence):
    if s == '_':
      temp+=1
    if temp == 0:
      bi_grams.append(sentence)
    elif temp == 2:
      tri_grams.append(sentence)
    elif temp == 4:
      qua_grams.append(sentence)



In [9]:
print(bi_grams)

['trợ_giảng', 'trợ_giảng', 'trợ_giảng', 'trú', 'trú', 'trú', 'nộp', 'nộp', 'nộp', 'rủi', 'rủi', 'rủi', 'đầu_nậu', 'đầu_nậu', 'đầu_nậu', 'nghĩa_vụ', 'nghĩa_vụ', 'nghĩa_vụ', 'nghĩa_vụ', 'nghĩa_vụ', '27', '27', 'trâu', 'trâu', 'trâu', 'trâu', 'pháo_đài_láng', 'pháo_đài_láng', 'pháo_đài_láng', 'pháo_đài_láng', 'dị_ứng', 'dị_ứng', 'nguyên', 'nguyên', 'nguyên', 'nguyên', 'nguyên', 'nguyên', 'hồn_ma', 'hồn_ma', 'hồn_ma', 'khắc_phục', 'khắc_phục', 'khắc_phục', 'khắc_phục', 'xảy', 'xảy', 'xảy', 'hoài_nghi', 'hoài_nghi', 'hoài_nghi', 'hoài_nghi', 'má', 'má', 'tđ', 'tđ', 'bún', 'bún', 'bún', 'lắc_đầu', 'lắc_đầu', 'lắc_đầu', 'vợ_chồng', 'vợ_chồng', 'ác_liệt', 'ác_liệt', 'bạn', 'bạn', 'bạn', 'la_ngà', 'la_ngà', '29', '29', 'tuyệt_vọng', 'tuyệt_vọng', 'tuyệt_vọng', 'tuyệt_vọng', 'tuyệt_vọng', 'lây', 'lây', 'lây', 'háo_hức', 'háo_hức', 'háo_hức', 'vụ_trưởng', 'vụ_trưởng', 'đoàn_tụ', 'đoàn_tụ', 'đoàn_tụ', 'đoàn_tụ', '17', '17', 'thành_hôn', 'thành_hôn', 'thành_hôn', 'thành_hôn', 'thành_hôn', 'dân_trí'

In [10]:
print(tri_grams)

['pháo_đài_láng', 'pháo_đài_láng', 'pháo_đài_láng', 'pháo_đài_láng', 'pháo_đài_láng', 'lắc_đầu_lè_lưỡi', 'lắc_đầu_lè_lưỡi', 'lắc_đầu_lè_lưỡi', 'nguyễn_thị_sinh', 'nguyễn_thị_sinh', 'nguyễn_thị_sinh', 'nguyễn_thị_sinh', 'nguyễn_thị_sinh', 'bùi_văn_mạnh', 'bùi_văn_mạnh', 'bùi_văn_mạnh', 'bùi_văn_mạnh', 'bùi_văn_mạnh', 'nguyễn_văn_thuật', 'nguyễn_văn_thuật', 'nguyễn_văn_thuật', 'nguyễn_văn_thuật', 'nguyễn_văn_thuật', 'nguyễn_văn_thuật', '20_-_25', '20_-_25', '20_-_25', 'nhà_đầu_tư', 'nhà_đầu_tư', 'nhà_đầu_tư', 'sea_games_22', 'sea_games_22', 'sea_games_22', 'nguyễn_văn_đẳng', 'nguyễn_văn_đẳng', 'nguyễn_văn_đẳng', 'nguyễn_văn_đẳng', 'nguyễn_văn_đẳng', 'viện_hàn_lâm', 'viện_hàn_lâm', 'viện_hàn_lâm', 'viện_hàn_lâm', 'dân_quân_tự_vệ', 'dân_quân_tự_vệ', 'dân_quân_tự_vệ', 'đào_xuân_nhung', 'đào_xuân_nhung', 'đào_xuân_nhung', 'đào_xuân_nhung', 'đào_xuân_nhung', 'đào_xuân_nhung', 'nguyễn_thanh_đoàn', 'nguyễn_thanh_đoàn', 'nguyễn_thanh_đoàn', 'nguyễn_thanh_đoàn', 'nguyễn_thanh_đoàn', 'hồ_chí_minh'

In [11]:
print(qua_grams)

['70_._000_-_150_._000', '70_._000_-_150_._000', '70_._000_-_150_._000', '70_._000_-_150_._000', '31_-_12_–_2004', '31_-_12_–_2004', '31_-_12_–_2004', '31_-_12_–_2004', '31_-_12_–_2004', '17_-_3_–_2004', '17_-_3_–_2004', '17_-_3_–_2004', '17_-_3_–_2004', '17_-_3_–_2004']


**5. SVM Classification**

In [12]:
# Tìm các bigram phổ biến nhất cho mỗi từ loại
for k in bigramFreq:
  maxFreq = 0
  maxPos = 'PROPN'
  for i, x in enumerate(bigramFreq[k]):
    if x > maxFreq:
      maxFreq = x
      maxPos = inverseTagsetDict[i]
    mostCommonBigrams[k] = maxPos

In [13]:
def Viterbi_rule_based(word,wordIdx, lineSize, line):
    """
    Hàm này tạo vector đặc trưng cho từng từ dựa trên quy tắc Viterbi.

    Parameters:
        word (str): Từ hiện tại.
        wordIdx (int): Vị trí của từ trong câu.
        lineSize (int): Độ dài của câu.
        line (list): Danh sách các từ trong câu.

    Returns:
        list: Vector đặc trưng cho từ.
    """
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0]
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "ADP":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        if posIdx >= len(tagsetDict):
           continue
        posIdx_array[posIdx] += 1.0/len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

In [14]:
sentences1 = []
first = 1
sentence1 = []
for line in train:
  try:
    i, word, pos = line.split('\n')[0].split('\t')
  except:
    continue
  i = int(i)
  if i == 1:
    if first == 1:
      first = 0
    else:
      sentences1.append(sentence1)
      sentence1 = []
  part = word + '/' + pos
  sentence1.append(part)

In [15]:
X_train = []
y_train = []
for line in sentences1:
  for i, w in enumerate(line):
    parts = w.split('/')
    word = parts[0]
    len_line = len(line)
    if len(parts) == 1 or parts[1] not in tagsetDict:
      continue
    try:
      y_train.append(wordBank[word][0])
      X_train.append(Viterbi_rule_based(word, i, len_line, line))
    except:
      continue

In [16]:
print (len(X_train))
print(len(y_train))
for i in range(10):
  print(i, ':', X_train[i])
print(y_train[:10])


39861
39861
0 : [1, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.015748031496062992, 0, 0, 0, 0.9842519685039354, 0, 0]
1 : [1, 0.05263157894736842, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9999999999999997, 0, 0, 0, 0, 0, 0, 0, 0]
2 : [1, 0.10526315789473684, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.05263157894736842, 0, 0.9473684210526312, 0, 0, 0, 0, 0, 0]
3 : [1, 0.15789473684210525, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9735682819383287, 0, 0, 0, 0, 0.02643171806167401, 0]
4 : [1, 0.21052631578947367, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0000000000000002, 0, 0, 0, 0, 0, 0]
5 : [1, 0.2631578947368421, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9999999999999994, 0, 0, 0, 0, 0, 0]
6 : [1, 0.3157894736842105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0]
7 : [1, 0.3684210526315789, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0000000000000002, 0, 0, 0, 0, 0, 0, 0, 0]
8 : [1, 0.42105263157894735, 0, 0, 0, 0, 0, 0, 0.015706806282722516, 0, 0, 0.9005235602094207, 0, 0, 0, 0, 0.08376963350785342, 0, 0, 0]
9 : [1, 0.47368421052631576, 0

In [17]:
sentences2 = []
first = 1
sentence2 = []
for line in test:

  i, word, pos = line.split('\n')[0].split('\t')
  i = int(i)
  if i == 1:
    if first == 1:
      first = 0
    else:
      sentences2.append(sentence2)
      sentence2 = []
  part = word + '/' + pos
  sentence2.append(part)

In [18]:
X_test = []
y_test = []
for line in sentences2:
  for i, w in enumerate(line):
    len_line = len(line)
    parts = w.split('/')
    if len(parts) == 1 or parts[1] not in tagsetDict or parts[1] == 'X':
      continue
    word = parts[0]
    pos = parts[1]
    if word in string.punctuation or word == ':.':
      continue
    X_test.append(Viterbi_rule_based(word, i, len_line, line))
    y_test.append(pos)


9. Test accuracy on subset of test data

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
model = OneVsRestClassifier(LinearSVC(random_state=0))

for epoch in range(10):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)

    train_precision = precision_score(y_train, y_pred_train, average='micro')
    val_precision = precision_score(y_val, y_pred_val, average='micro')

    train_recall = recall_score(y_train, y_pred_train, average='micro')
    val_recall = recall_score(y_val, y_pred_val, average='micro')

    print(f"Epoch {epoch+1}:")
    print(f"  Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Train Precision: {train_precision:.4f}, Validation Precision: {val_precision:.4f}")
    print(f"  Train Recall: {train_recall:.4f}, Validation Recall: {val_recall:.4f}")

with open('postagging_SVM_Treebank.pkl', 'wb') as f:
    pickle.dump(model, f)


Epoch 1:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation Recall: 0.9401
Epoch 2:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation Recall: 0.9401
Epoch 3:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation Recall: 0.9401
Epoch 4:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation Recall: 0.9401
Epoch 5:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation Recall: 0.9401
Epoch 6:
  Train Accuracy: 0.9374, Validation Accuracy: 0.9401
  Train Precision: 0.9374, Validation Precision: 0.9401
  Train Recall: 0.9374, Validation R

In [20]:
with open('postagging_SVM_Treebank.pkl', 'rb') as f:
    model = pickle.load(f)

y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='micro')
test_recall = recall_score(y_test, y_pred_test, average='micro')

print("\nFinal Evaluation on Test Set:")
print(f"  Test Accuracy: {test_accuracy:.4f}")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall: {test_recall:.4f}")


Final Evaluation on Test Set:
  Test Accuracy: 0.7778
  Test Precision: 0.7778
  Test Recall: 0.7778


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

         ADJ       0.58      0.82      0.68       504
         ADP       0.86      0.76      0.81       729
         ADV       0.78      0.90      0.83       886
         AUX       0.99      0.69      0.82       344
       CCONJ       1.00      0.88      0.94       163
         DET       0.90      0.12      0.21      1193
        INTJ       0.35      1.00      0.52         6
        NOUN       0.85      0.91      0.88      2842
         NUM       0.86      0.94      0.90       246
        PART       0.52      0.58      0.55        79
        PRON       0.94      0.90      0.92       555
       PROPN       0.40      0.84      0.54       268
       SCONJ       0.76      0.94      0.85       309
         SYM       1.00      0.25      0.40         8
        VERB       0.73      0.86      0.79      1805

    accuracy                           0.78      9937
   macro avg       0.77      0.76      0.71      9937
weighted avg       0.81   

Test with IOT Dataset

In [34]:
with open('postagging_SVM_Treebank.pkl', 'rb') as f:
    model = pickle.load(f)

f_iot_test = io.open("IOT_test.txt", encoding='utf-8').readlines()
iot_test = f_iot_test[:100]

X_iot_test = []
y_iot_test = []
for line in iot_test:
  l_split = line.split()
  for i, w in enumerate(l_split):
    len_line = len(l_split)
    parts = w.split('/')
    if len(parts) == 1:
      continue
    word = parts[0].lower()
    pos = parts[1]
    if word in string.punctuation or word == ':.':
      continue
    try:
      X_iot_test.append(Viterbi_rule_based(word, i, len_line, line))
      y_iot_test.append(pos)
    except:
      continue

y_pred_test = model.predict(X_iot_test)
test_accuracy = accuracy_score(y_iot_test, y_pred_test)
test_precision = precision_score(y_iot_test, y_pred_test, average='micro')
test_recall = recall_score(y_iot_test, y_pred_test, average='micro')

print("\nFinal Evaluation on IOT Test Set:")
print(f"  Test Accuracy: {test_accuracy:.4f}")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall: {test_recall:.4f}")


Final Evaluation on IOT Test Set:
  Test Accuracy: 0.6466
  Test Precision: 0.6466
  Test Recall: 0.6466


# ***7. Using the Longest Matching algorithm***

In [35]:
import unicodedata as ud
import re

def syllablize(sentence):
  """
    Hàm này chia câu thành các âm tiết.

    Parameters:
        sentence (str): Câu cần chia.

    Returns:
        list: Danh sách các âm tiết trong câu.
    """
  word = '\w+'
  non_word = '[^\w\s]'
  digits = '\d+([\.,_]\d+)+'

  patterns = []
  patterns.extend([word, non_word, digits])
  patterns = f"({'|'.join(patterns)})"

  sentence = ud.normalize('NFC', sentence)
  tokens = re.findall(patterns, sentence, re.UNICODE)
  return [token[0] for token in tokens]

In [36]:
def longest_matching(sentence, bi_grams, tri_grams):
  """
    Hàm này tìm các từ ghép dài nhất trong câu.

    Parameters:
        sentence (str): Câu cần kiểm tra.
        bi_grams (list): Danh sách các bi-grams.
        tri_grams (list): Danh sách các tri-grams.

    Returns:
        list: Danh sách các từ ghép dài nhất trong câu.
    """
  syllables = syllablize(sentence)
  syl_len = len(syllables)

  curr_id = 0
  word_list = []
  done = False

  while (curr_id < syl_len) and (not done):
    curr_word = syllables[curr_id]
    if curr_id >= syl_len - 1:
      word_list.append(curr_word)
      done = True
    else:
      next_word = syllables[curr_id + 1]
      pair_word = " ".join([curr_word.lower(), next_word.lower()])
      if curr_id >=(syl_len - 2):
        if pair_word in bi_grams:
          word_list.append("_".join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
      else:
        next_next_word = syllables[curr_id + 2]
        triple_word = ' '.join([pair_word, next_next_word.lower()])
        if triple_word in tri_grams:
          word_list.append('_'.join([curr_word, next_word, next_next_word]))
          curr_id += 3
        elif pair_word in bi_grams:
          word_list.append('_'.join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
  return word_list

8. Conver into String

In [38]:
def toString(wl, model):
  """
    Hàm này chuyển đổi danh sách các từ thành chuỗi.

    Parameters:
        wl (list): Danh sách các từ cần chuyển đổi.

    Returns:
        str: Chuỗi kết quả.
    """
  wl = longest_matching(wl, bi_grams, tri_grams)
  X = []
  A = []
  text = ''
  for i in set(wl):
    if i == '.':
      A = './.'
    else:
      X.append(Viterbi_rule_based(i, 1, 1, 1))
      A = str(model.predict(X))
  for i in range(len(wl)):
    text += wl[i]
    text +='/'
    text +=str(A[2])
    text += ' '

  return wl[i], str(A[2])

10. Check how a sentence is tagged by the POS tagger

In [42]:
if __name__ == "__main__":
    #wr = input('Text: ')
    with open('postagging_SVM_Treebank.pkl', 'rb') as f:
      model = pickle.load(f)
    wr ="""bật đèn phòng khách, à không hãy bật quạt đi nào""".lower()
    print(wr)
    result = {'N': [], 'V': []}
    wl = ViTokenizer.tokenize(wr)
    wl = wl.split()
    for i in wl:
      try:
          word, pos =  toString(i, model)
          result[pos].append(word)
      except Exception:
          #print(i,"X",sep='/',end='')
          #print(' ',end='')
          pass
    print(result)

bật đèn phòng khách, à không hãy bật quạt đi nào
{'N': ['đèn', 'phòng', 'khách'], 'V': ['bật', 'bật', 'đi']}
